In [ ]:
import torch
import pandas as pd
import numpy as np
from torch import nn
import os
import sklearn
import matplotlib.pyplot as plt
import sys
import re

### For Colab

In [ ]:
# # This mounts your Google Drive to the Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')

# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# # assignment folder, e.g. 'cs231n/assignments/assignment1/'
# FOLDERNAME = 'PyTorch Test Project/N_TXT'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# # This downloads the CIFAR-10 dataset to your Drive
# # if it doesn't already exist.
# # %cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
# # !bash get_datasets.sh
# # %cd /content/drive/My\ Drive/$FOLDERNAME

In [ ]:
# path = "/content/drive/MyDrive/PyTorch Test Project/N_TXT"

# os.chdir(path)

# fx_data = np.array([], dtype=np.float32)

# #gets data from file specified by file_path
# def get_contents(file_path):
#     global fx_data
#     skip_lines = 7

#     with open(file_path,'r') as file:

#         for skips in range(skip_lines):
#             next(file)

#         lines = file.readlines()[:-1]

#         for line in lines:
#             data = line.split()[-1]
#             fx_data = np.append(fx_data, np.float32(data))
#             #print(data)

In [ ]:
path = "C:\\Users\Admin\\.vscode\\PyTorch\\Time Series Prediction\\N_TXT"

os.chdir(path)

fx_data = np.array([], dtype=np.float32)

#gets data from file specified by file_path
def get_contents(file_path):
    global fx_data
    skip_lines = 7

    with open(file_path,'r') as file:

        for skips in range(skip_lines):
            next(file)

        lines = file.readlines()[:-1]

        for line in lines:
            data = line.split()[-1]
            fx_data = np.append(fx_data, np.float32(data))
            #print(data)

In [ ]:
#go through all files in directory and extract data from txt files
for file in os.listdir():
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        

        print(f"Currently working on: {file_path}")
        get_contents(file_path)

fx_data.dtype

In [ ]:
#create time "sample" for dataframe
time_data = np.array([], dtype=np.float32)

for x in range(len(fx_data)):
    time_data = np.append(time_data, np.float32(x))
    type(x)

time_data, time_data[0].dtype

In [ ]:
#create time "sample" for dataframe
dataset = pd.DataFrame({"Time": time_data,
                        "FX Channel": fx_data})
dataset, dataset.size

In [ ]:
plt.plot(dataset["Time"], dataset["FX Channel"]), type(dataset)

In [ ]:
X = dataset["Time"]
y = dataset["FX Channel"]

X

In [ ]:
#trying to split data loader now? not too sure if this is the best way
split_idx = int(0.8 * len(dataset))

temp_X_train = X[:split_idx]
temp_X_test = X[split_idx:]

temp_y_train = y[:split_idx]
temp_y_test = y[split_idx:]

temp_X_train.shape, temp_X_test.shape, temp_y_train.shape, temp_y_test.shape, len(X), temp_X_train

In [ ]:
def split_data(data, nsteps):
    data_list = list()

    for i in range(len(data)):
        end = i + nsteps
        if(end > len(data-1)):
            break

        extracted_data = data[i:end]

        data_list.append(extracted_data)

    return np.array(data_list)

In [ ]:
len(temp_y_train), y[0:8]

In [ ]:
nsteps = 3
X_train = split_data(temp_X_train, nsteps)
X_test = split_data(temp_X_test, nsteps)
y_train = split_data(temp_y_train, nsteps)
y_test = split_data(temp_y_test, nsteps)

X_train, type(X_train), len(X_train)

In [ ]:
y_train, len(y_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

X_train.shape, y_train.shape, X_train.max(), X_train.min()

In [ ]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
type(X_train)

In [ ]:
# X_train = X_train.unsqueeze(dim = 2)
# X_test = X_test.unsqueeze(dim = 2)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#set device
device = "cuda" if torch.cuda.is_available() else "cpu"

device

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label

train_dataset = TimeSeriesDataset(X_train.reshape(X_train.shape[0],X_train.shape[1],1),y_train)
test_dataset = TimeSeriesDataset(X_test.reshape(X_test.shape[0],X_test.shape[1],1),y_test)

In [ ]:
type(train_dataset)

In [ ]:
from torch.utils.data import DataLoader

#setup batch size hyperparameter
BATCH_SIZE = 32

#turn datasets into iterables (batches)
train_loader = DataLoader(dataset=train_dataset, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True,
                              drop_last=True)

test_loader = DataLoader(dataset=test_dataset, 
                              batch_size=BATCH_SIZE, 
                              shuffle=False, 
                              drop_last=True)

#check out what we've created
print(f"Dataloaders: {train_loader, test_loader}")
print(f"Length of train_dataloader: {len(train_loader)} batches of {BATCH_SIZE}")
print(f"Length of train_dataloader: {len(test_loader)} batches of {BATCH_SIZE}")

In [ ]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

In [ ]:
#set manual seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device

In [ ]:
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32, 50)  # Adjust based on the output size of the last conv layer
        self.fc2 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CNN1D()

In [ ]:
model = CNN1D().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

print(model)

In [ ]:
device

In [ ]:
train_losses = []
valid_losses = []

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for batch_data, batch_targets in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_data.float().to(device))
        loss = criterion(outputs.float(), batch_targets.type(torch.float32).to(device))

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
X_test = torch.from_numpy(X_test).float()

In [ ]:
model.eval()
with torch.inference_mode():
    test_pred = model(X_test.type(torch.float32).to(device)).to('cpu').numpy()

plt.plot(y_test, label='Actual FX')
plt.plot(test_pred.squeeze(), label='Test Predicted FX')
plt.xlabel('Time')
plt.ylabel('FX Channel')
plt.legend()
plt.show()

In [ ]:
import random
random.seed(42)
test_samples = []
test_labels = []
for sample, label in (test_dataset):

    test_samples.append(torch.from_numpy(sample).float())
    test_labels.append(torch.from_numpy(label).float())

#view first sample sehape
len(test_samples), test_samples[0].shape, type(test_samples[0])

In [ ]:
def make_predictions(model: torch.nn.Module,
                     data:list,
                     device: torch.device = device):
    predictions_list = []
    model.to(device)
    model.eval()
    with torch.inference_mode():
        for sample in data:
            print(type(sample))
            sample = sample.to(device)

            prediction = model(sample)

            #move pred probs to cpu
            predictions_list.append(prediction.cpu())

    #stack turns list into a tensor
    return torch.stack(predictions_list)

In [ ]:
test_samples[0].shape

In [ ]:
#make predictions
pred_probs = make_predictions(model=model,
                              data=test_samples)

pred_probs[0].shape

In [ ]:
data = np.random.rand(1000, 50)
targets = np.random.rand(1000, 1)

# Convert to PyTorch tensors
data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(1)

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(data_tensor).squeeze().numpy()

# Plot predictions vs true labels
plt.figure(figsize=(10, 6))
plt.plot(targets, label='True Labels')
plt.plot(predictions, label='Predictions', alpha=0.7)
plt.xlabel('Sample')
plt.ylabel('Value')
plt.title('True Labels vs Predictions')
plt.legend()
plt.show()

In [ ]:

def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs,labels) in enumerate(train_loader):

        inputs = inputs.type((torch.float32)).to(device)
        # print(f"shape: {inputs.shape}")
        labels = labels.type(torch.float32).to(device)
        # print(inputs)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.detach().cpu().numpy())
    
    print(f'train_loss {train_loss}')
    
def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.inference_mode():
        for idx, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.type(torch.float32).to(device)
            labels = labels.type(torch.float32).to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(test_loader)
        valid_losses.append(valid_loss.detach().cpu().numpy())
        print(f'valid_loss {valid_loss}')

In [ ]:
import gc

epochs = 200
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    Valid()
    gc.collect()

In [ ]:
X_train.shape, type(X_train)

In [ ]:
# plt.plot(train_acc, label='Train Acc')
# plt.plot(test_acc, label='Test Acc')
# plt.xlabel('Time')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
target_x = split_data(X, nsteps)
target_y = split_data(y, nsteps)

In [ ]:
len(target_x)

In [ ]:
type(target_x)

In [ ]:
inputs = torch.from_numpy(target_x.transpose(0, 1))
torch.reshape(inputs, (len(target_x), 3, 1))

In [ ]:
len(inputs), len(target_x), (target_x.shape), inputs.shape[0]

In [ ]:
model.eval()
prediction = []
batch_size = 1
iterations =  int(inputs.shape[0]/1)
print(iterations)


In [ ]:
print(iterations-batch_size)

In [ ]:
for i in range(iterations-batch_size):
    preds = model(torch.tensor(inputs[i:i + batch_size].transpose(0, 1)).to(device).float())
    # print(preds.squeeze().shape)
    # prediction.append(preds.detach().squeeze().cpu().numpy())
    prediction.append(preds.cpu().squeeze().detach().numpy())
    # break

print(prediction[0])

In [ ]:
fig, ax = plt.subplots(1, 2,figsize=(11,4))
ax[0].set_title('predicted')
ax[0].plot(X,prediction)
ax[1].set_title('true')
ax[1].plot(target_y)
plt.show()

In [ ]:
X_test.transpose(dim0=1, dim1=-1).shape

In [ ]:
prediction = []
model.eval()
with torch.inference_mode():
    for idx, (inputs, labels) in enumerate(test_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                preds = model(inputs.float())
                prediction.append(preds.detach().cpu().numpy())               

In [ ]:
type(prediction), prediction[1][2], len(test_loader)

In [ ]:
plt.plot(target_y, label='Actual FX')
plt.plot(prediction, label='Test Predicted FX')
plt.xlabel('Time')
plt.ylabel('FX Channel')
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.inference_mode():
    test_pred = model(X_test.transpose(dim0=0, dim1=-1).to(device)).to('cpu').numpy()

plt.plot(y_test, label='Actual FX')
plt.plot(test_pred.squeeze(), label='Test Predicted FX')
plt.xlabel('Time')
plt.ylabel('FX Channel')
plt.legend()
plt.show()

In [ ]:
test_pred.squeeze().shape

In [ ]:
X_test.shape, X_test[0]

In [ ]:
temp = torch.arange(0, 1, 0.000016)
temp = temp.unsqueeze(dim=1)
temp.shape

In [ ]:
model.eval()
with torch.inference_mode():
    test_pred = model(temp.unsqueeze(dim=1)).to('cpu').numpy()

# plt.xlim(0, (10000))
# plt.ylim(-0, 1)
# plt.autoscale(False)
plt.plot(y_test, label='Actual FX')
plt.plot(test_pred, label='Test Predicted FX')
plt.xlabel('Time')
plt.ylabel('FX Channel')
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.inference_mode():
    test_pred = model(X_test.to(device)).to('cpu').numpy()

plt.plot(y_test, label='Actual FX')
plt.plot(test_pred, label='Test Predicted FX')
plt.xlabel('Time')
plt.ylabel('FX Channel')
plt.legend()
plt.show()

In [ ]:
y_test[0], y_test[5800], y_train[0], y_train[23000]

In [ ]:
X_test.max(), X_test.min()